In [68]:
from pyspark.sql import SparkSession
import datetime as dt
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('etl').getOrCreate()
df = spark.read.parquet('data/catalog.parquet')

## Measurement-mapping

TODO:

[X] Rename Columns  
[X] Split Datetime  
[ ] Combine blood pressure components  
[ ] Check all required fields mapped

In [95]:
filtered = df.filter(df['resourceType'] == 'Observation')

In [99]:
Measurement = filtered.filter(filtered.valueCodeableConcept.isNull())
Observation = filtered.filter(filtered.valueCodeableConcept.isNotNull())
Measurement = Measurement.select(['id',
                            'subject',
                            'code',
                            'performer',
                            'encounter',
                            'meta',
                            'valueQuantity',
                            'effectiveDateTime',
                            'Extension.valueCodeableConcept',
                            'component'])
#Measurement.printSchema()
#Measurement.toPandas().to_csv("dfdfa.csv", header=True)

##################


split_dates = F.split(Measurement["effectiveDateTime"], 'T')

combined_blood_pressures = F.coalesce(Measurement.component.getItem(0).valueQuantity, Measurement.component.getItem(1).valueQuantity)


 #      .withColumn("valueQuantity", combined_blood_pressures)\
Measurement = Measurement.withColumnRenamed("id", "measurement_id")\
                         .withColumn("valueQuantityNew", combined_blood_pressures)\
                         .withColumn("measurement_date", split_dates.getItem(0))\
                         .withColumn("person_id", Measurement.subject.reference)\
                         .drop("subject")\
                         .withColumnRenamed("code", "measurement_concept_id")\
                         .withColumnRenamed("effectiveDateTime", "measurement_datetime")\
                         .drop("valueCodeableConcept")\
                         .withColumnRenamed("category", "measurement_type_concept_id")\
                         .withColumn("value_as_number", Measurement.valueQuantity.value)\
                         .withColumn("visit_occurrence_id", Measurement.encounter.reference)\
                         .drop("encounter")\
                         .withColumn("unit_source_value", Measurement.valueQuantity.unit)\
                         .withColumnRenamed("performer", "provider_id")\
                         .drop("valueQuantity")


Measurement.toPandas().to_csv("measurememts.csv", header=True)

Measurement.show(1)

+--------------------+----------------------+-----------+--------------------+--------------------+---------+----------------+----------------+--------------------+---------------+--------------------+-----------------+
|      measurement_id|measurement_concept_id|provider_id|                meta|measurement_datetime|component|valueQuantityNew|measurement_date|           person_id|value_as_number| visit_occurrence_id|unit_source_value|
+--------------------+----------------------+-----------+--------------------+--------------------+---------+----------------+----------------+--------------------+---------------+--------------------+-----------------+
|3a85e0c5-c469-42c...|  {[{http://loinc.o...|       null|{2020-12-15T20:38...|2011-12-01T03:37:...|     null|            null|      2011-12-01|urn:uuid:46c99701...|{0.13747, null}|urn:uuid:ef55a399...|             kU/L|
+--------------------+----------------------+-----------+--------------------+--------------------+---------+-----------